In [40]:
import pinocchio as pin
from pinocchio.utils import *

import example_robot_data as robex
import numpy as np
from numpy.linalg import inv, pinv, eig, norm, svd, det

import vizutils
import random
import math


In [12]:
robot = robex.loadSolo()

#Viewer = pin.visualize.GepettoVisualizer
Viewer = pin.visualize.MeshcatVisualizer

viz = Viewer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(loadModel=True)


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [13]:
hasattr(viz.viewer, 'jupyter_cell') and viz.viewer.jupyter_cell()


In [27]:
viz.display(robot.q0)

vizutils.addViewerBox(viz, 'world/box', .01, 1.45, .01, [1., .2, 1, 1])
vizutils.applyViewerConfiguration(viz, 'world/box', [0, -0, .01, 1, 0, 0, 0])

vizutils.addViewerBox(viz, 'world/floor', 12, 12, .1, [.8, .8, .8, .41])
vizutils.applyViewerConfiguration(viz, 'world/floor', [0, 0, -.01, 1, 0, 0, 0])


## RRT*


In [30]:
#constants
XDIM = 640
YDIM = 480
WINSIZE = [XDIM, YDIM]
EPSILON = 7.0
NUMNODES = 5000
RADIUS=15

### Utility Functions

In [34]:
def dist(p1,p2):
    return sqrt((p1[0]-p2[0])*(p1[0]-p2[0])+(p1[1]-p2[1])*(p1[1]-p2[1]
))


def step_from_to(p1,p2):
    if dist(p1,p2) < EPSILON:
        return p2
    else:
        theta = atan2(p2[1]-p1[1],p2[0]-p1[0])
        return p1[0] + EPSILON*cos(theta), p1[1] + EPSILON*sin(theta)


def chooseParent(nn,newnode,nodes):
     for p in nodes:
        if dist([p.x,p.y],[newnode.x,newnode.y]) <RADIUS and p.cost+dist([p.x,p.y],[newnode.x,newnode.y]) < nn.cost+dist([nn.x,nn.y],[newnode.x,newnode.y]):
            nn = p
        newnode.cost=nn.cost+dist([nn.x,nn.y],[newnode.x,newnode.y])
        newnode.parent=nn
        return newnode,nn



In [36]:
def reWire(nodes,newnode,pygame,screen):
    white = 255, 240, 200
    black = 20, 20, 40
    for i in range(len(nodes)):
        p = nodes[i]
        if p!=newnode.parent and dist([p.x,p.y],[newnode.x,newnode.y]) <RADIUS and newnode.cost+dist([p.x,p.y],[newnode.x,newnode.y]) < p.cost:
            #pygame.draw.line(screen,white,[p.x,p.y],[p.parent.x,p.parent.y])  
            p.parent = newnode
            p.cost=newnode.cost+dist([p.x,p.y],[newnode.x,newnode.y]) 
            nodes[i]=p  
              
            ## DRAW! pygame.draw.line(screen,black,[p.x,p.y],[newnode.x,newnode.y])                    
    return nodes

In [37]:
def drawSolutionPath(start,goal,nodes,pygame,screen):
    pink = 200, 20, 240
    nn = nodes[0]
    for p in nodes:
        if dist([p.x,p.y],[goal.x,goal.y]) < dist([nn.x,nn.y],[goal.x,goal.y]):
            nn = p
    while nn!=start:
        #pygame.draw.line(screen,pink,[nn.x,nn.y],[nn.parent.x,nn.parent.y],5)  
        nn=nn.parent

In [38]:
class Node:
    x = 0
    y = 0
    cost=0  
    parent=None
    def __init__(self,xcoord, ycoord):
        self.x = xcoord
        self.y = ycoord

In [41]:
def main():
    #initialize and prepare screen
    #pygame.init()
    #screen = pygame.display.set_mode(WINSIZE)
    #pygame.display.set_caption('RRTstar')
    white = 255, 240, 200
    black = 20, 20, 40
    #screen.fill(white)

    nodes = []
    
    #nodes.append(Node(XDIM/2.0,YDIM/2.0)) # Start in the center
    nodes.append(Node(0.0,0.0)) # Start in the corner
    start=nodes[0]
    goal=Node(630.0,470.0)
    
    for i in range(NUMNODES):
        rand = Node(random.random()*XDIM, random.random()*YDIM)
        nn = nodes[0]
        for p in nodes:
            if dist([p.x,p.y],[rand.x,rand.y]) < dist([nn.x,nn.y],[rand.x,rand.y]):
                nn = p
        interpolatedNode= step_from_to([nn.x,nn.y],[rand.x,rand.y])
    
    newnode = Node(interpolatedNode[0],interpolatedNode[1])
    [newnode,nn]=chooseParent(nn,newnode,nodes);
       
    nodes.append(newnode)
    pygame.draw.line(screen,black,[nn.x,nn.y],[newnode.x,newnode.y])
        nodes=reWire(nodes,newnode,pygame,screen)
        pygame.display.update()
        #print i, "    ", nodes

        for e in pygame.event.get():
            if e.type == QUIT or (e.type == KEYUP and e.key == K_ESCAPE):
                sys.exit("Leaving because you requested it.")
                
    drawSolutionPath(start,goal,nodes,pygame,screen)
    pygame.display.update()
# if python says run, then we should run

IndentationError: expected an indented block (<ipython-input-41-c1c5e44ab96b>, line 18)